In [ ]:
'''test = pd.DataFrame()
test ["Survived"] = clean_sharks[clean_sharks["Fatal"]=="N"].groupby(["Sex"]).count()["Fatal"]
test ["Died"] = clean_sharks[clean_sharks["Fatal"]=="Y"].groupby(["Sex"]).count()["Fatal"]
test ["Surv rate"] = (test["Survived"]/(test["Survived"]+test["Died"]))*100
test
'''

In [ ]:
'''
test2=pd.DataFrame()
test2["Reported attacks"] = clean_sharks.groupby(["Country"]).count()["Year"]
test2["Common Sex"]=clean_sharks.groupby(["Country"])["Sex"].agg(pd.Series.mode)
test2["Victim age"]=clean_sharks[clean_sharks["Sex"]==test2["Sex"]].groupby(["Country"])["Age"].agg(pd.Series.mean)
test2 = test2.sort_values("Reported attacks",ascending=False)
test2
#clean_sharks[clean_sharks["Country"]=="USA"]["Sex"].mode()[0]
#test2["Sex"]==clean_sharks["Sex"]
'''